In [1]:
import cv2
import numpy as np
import glob

In [2]:
dim = 20 #mm
ratio = (8,5) #chessboard x,y
path = "./res/files_camera_calib"

object_points = np.zeros((ratio[0] * ratio[1], 3), np.float32)
object_points[:, :2] = np.mgrid[0:ratio[0], 0:ratio[1]].T.reshape(-1, 2)
object_points *= dim

objpoints = []  
imgpoints = []  

images = glob.glob(path + "/*.jpg")

In [3]:
for im in images:
    img =  cv2.cvtColor(cv2.imread(im),cv2.COLOR_BGR2GRAY) 
    cv2.imshow('img', img)
    cv2.waitKey(100)
    cv2.destroyAllWindows()

In [4]:
for fname in images:
    img = cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2GRAY) 
    ret, corners = cv2.findChessboardCorners(img, ratio, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK)
    print(ret)
    if ret:
        corners_refined = cv2.cornerSubPix(img,corners,(11, 11), (-1,-1),criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        imgpoints.append(corners_refined)
        objpoints.append(object_points)
        cv2.drawChessboardCorners(img, ratio, corners_refined, ret)
        cv2.imshow('Chessboard Corners', img)
        cv2.waitKey(100)
        

True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True


In [12]:
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[::-1], None, None)
print(dist_coeffs)

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'calibrateCamera'
> Overload resolution failed:
>  - Can't parse 'imageSize'. Expected sequence length 2, got 3
>  - Can't parse 'imageSize'. Expected sequence length 2, got 3


In [6]:
print(np.array([[279.35,0,311.76],[0,371.99,240.00],[0,0,1]],np.float64))

[[279.35   0.   311.76]
 [  0.   371.99 240.  ]
 [  0.     0.     1.  ]]


In [7]:
import time
for im in glob.glob(path + "/val/*.jpg"):
    img = cv2.imread(im)
    h, w = img.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))
    
    # Undistort image
    undistorted_img = cv2.undistort(img, camera_matrix, dist_coeffs, None, new_camera_matrix)
    
    # Crop the image to the region of interest
    x, y, w, h = roi
    undistorted_img = undistorted_img[y:y+h, x:x+w]
    cv2.imshow('Undistorted Image', undistorted_img)
   

In [8]:
cv2.destroyAllWindows()
time.sleep(2)

[[286.98062406   0.         311.41898239]
 [  0.         382.1315285  241.78807416]
 [  0.           0.           1.        ]]
